# Iterative Phase Estimation

In [1]:
from qiskit import QuantumCircuit, Aer, assemble, execute
from math import pi, sqrt
import numpy as np
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.quantum_info import Statevector as sv
from qiskit.circuit.library.standard_gates import RZGate
import itertools

In [2]:
def output_counts(quantum_circuit):
    simulator = Aer.get_backend('qasm_simulator')
    result = execute(quantum_circuit, simulator).result()
    counts = result.get_counts(quantum_circuit)
    return counts

In [3]:
def phase_estimation_iterate(iteration, c_unitary_gate, bits):
    iteration_circuit = QuantumCircuit(2, 1)
    iteration_circuit.h(0) # generates superposition state in 1st qbit
    iteration_circuit.x(1) # generates |1> in 2nd qbit
    
    factor = 4
    correction = 0 # generates the correction fraction depending on pre computed bits
    for bit in bits:
        correction += bit / factor
        factor *= 2
    
    # generate all the control U gates
    n = 2 ** (iteration - 1)
    for i in range(int(n)):
        c_unitary_gate(iteration_circuit, 0, 1)
    
    # add the correction
    iteration_circuit.rz(- 2 * pi * correction, 0)
    
    # convert back to computational basis
    iteration_circuit.h(0)
    
    # measure the qbit
    iteration_circuit.measure(0, 0)

    result = output_counts(iteration_circuit)
    
    if '0' in result.keys():
        bits = [0] + bits
    else:
        bits = [1] + bits
    # last "iteration" bits  
    return bits

In [4]:
def cu5piby8(circuit, control, target):
    circuit.cp(5 * pi / 8, control, target)

In [5]:
def cu5piby4(circuit, control, target):
    circuit.cp(5 * pi / 4, control, target)

In [6]:
def iterative_phase_estimation(c_unitary_gate, precision):
    bits = []
    for iteration in range(precision, 0, -1):
        bits = phase_estimation_iterate(iteration, c_unitary_gate, bits)
    
    factor = 2
    fraction = 0
    for bit in bits:
        fraction += bit / factor
        factor *= 2
    
    return "0." + "".join(list(map(str, bits))), fraction

In [7]:
iterative_phase_estimation(cu5piby8, 4)

('0.0101', 0.3125)

In [8]:
iterative_phase_estimation(cu5piby4, 3)

('0.101', 0.625)